In [40]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [41]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [42]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(6973, 8)

In [43]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6680, 5)

In [44]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6979, 10)

fin empresas y comienzo de rescate de pacientes


In [45]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1635688, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [46]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1210074, 4)

In [47]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1210074, 13)

In [49]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [50]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [51]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [52]:
#pacientes_empresas22.shape

In [53]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [54]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [55]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [71]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Colmena"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [72]:
Pacientes_Archivo.head(10)

,Rut Titular,DV Titular,Rut B.,DV B.,rut1,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,sexo,nacimiento,Fecha Suscripción,Fecha Inicio B.,Monto Prima
0,16644184-6,6,16644184-6,6,16644184-6,AFILIADO,TERESA DE,LAGOS,SALDIVIA,LAGOS SALDIVIA,COTIZANTE,F,1987-08-14,2022-09-08,2022-11-01,0.1624
1,2589337-9,9,2589337-9,9,2589337-9,AFILIADO,GREGORIA,VELASCO,MINGUEZ,VELASCO MINGUEZ,COTIZANTE,F,1927-10-04,2021-11-10,2022-01-01,0.1624
2,4182163-9,9,4182163-9,9,4182163-9,AFILIADO,LUZ MARIA,SANHUEZA,GONZALEZ,SANHUEZA GONZALEZ,COTIZANTE,F,1937-12-16,2022-07-12,2022-09-01,0.1624
3,5289238-4,4,5289238-4,4,5289238-4,AFILIADO,DOMINGO,TORRES,TORREALBA,TORRES TORREALBA,COTIZANTE,M,1945-09-23,2022-02-09,2022-04-01,0.1624
4,5320398-1,1,5320398-1,1,5320398-1,AFILIADO,MARIO ADOLFO,ARIZTIA,MONTANE,ARIZTIA MONTANE,COTIZANTE,M,1945-02-20,2021-10-29,2021-12-01,0.1624
5,9638895-0,0,9638895-0,0,9638895-0,AFILIADO,CORINA,CLARO,COLLADO,CLARO COLLADO,COTIZANTE,F,1965-09-25,2022-03-15,2022-05-01,0.1624
6,11444946-6,6,11444946-6,6,11444946-6,AFILIADO,ROBERTO,NEIRA,MARCHANT,NEIRA MARCHANT,COTIZANTE,M,1969-07-22,2021-04-23,2021-06-01,0.1624
7,12618267-8,8,12618267-8,8,12618267-8,AFILIADO,CRISTIAN,GALAZ,CAMPUSANO,GALAZ CAMPUSANO,COTIZANTE,M,1974-09-25,2021-04-04,2021-06-01,0.1624
8,13112696-4,4,13112696-4,4,13112696-4,AFILIADO,YOHANA,BELTRAN,HERRERA,BELTRAN HERRERA,COTIZANTE,F,1976-06-26,2022-08-29,2022-10-01,0.1624
9,14213917-0,0,14213917-0,0,14213917-0,AFILIADO,PATRICIO,JORQUERA,CARTES,JORQUERA CARTES,COTIZANTE,M,1980-12-22,2021-04-22,2021-06-01,0.1624


In [73]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [74]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Rut Titular,DV Titular,Rut B.,DV B.,rut1,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,sexo,nacimiento,Fecha Suscripción,Fecha Inicio B.,Monto Prima,rut
0,16644184-6,6,16644184-6,6,16644184-6,AFILIADO,TERESA DE,LAGOS,SALDIVIA,LAGOS SALDIVIA,COTIZANTE,F,1987-08-14,2022-09-08,2022-11-01,0.1624,166441846
1,2589337-9,9,2589337-9,9,2589337-9,AFILIADO,GREGORIA,VELASCO,MINGUEZ,VELASCO MINGUEZ,COTIZANTE,F,1927-10-04,2021-11-10,2022-01-01,0.1624,25893379
2,4182163-9,9,4182163-9,9,4182163-9,AFILIADO,LUZ MARIA,SANHUEZA,GONZALEZ,SANHUEZA GONZALEZ,COTIZANTE,F,1937-12-16,2022-07-12,2022-09-01,0.1624,41821639
3,5289238-4,4,5289238-4,4,5289238-4,AFILIADO,DOMINGO,TORRES,TORREALBA,TORRES TORREALBA,COTIZANTE,M,1945-09-23,2022-02-09,2022-04-01,0.1624,52892384
4,5320398-1,1,5320398-1,1,5320398-1,AFILIADO,MARIO ADOLFO,ARIZTIA,MONTANE,ARIZTIA MONTANE,COTIZANTE,M,1945-02-20,2021-10-29,2021-12-01,0.1624,53203981


In [75]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21772\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Rut Titular,DV Titular,Rut B.,DV B.,rut1,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,sexo,nacimiento,Fecha Suscripción,Fecha Inicio B.,Monto Prima,rut
0,16644184-6,6,16644184-6,6,16644184-6,AFILIADO,TERESA DE,LAGOS,SALDIVIA,LAGOS SALDIVIA,COTIZANTE,F,1987-08-14,2022-09-08,2022-11-01,0.1624,166441846
1,2589337-9,9,2589337-9,9,2589337-9,AFILIADO,GREGORIA,VELASCO,MINGUEZ,VELASCO MINGUEZ,COTIZANTE,F,1927-10-04,2021-11-10,2022-01-01,0.1624,25893379
2,4182163-9,9,4182163-9,9,4182163-9,AFILIADO,LUZ MARIA,SANHUEZA,GONZALEZ,SANHUEZA GONZALEZ,COTIZANTE,F,1937-12-16,2022-07-12,2022-09-01,0.1624,41821639
3,5289238-4,4,5289238-4,4,5289238-4,AFILIADO,DOMINGO,TORRES,TORREALBA,TORRES TORREALBA,COTIZANTE,M,1945-09-23,2022-02-09,2022-04-01,0.1624,52892384
4,5320398-1,1,5320398-1,1,5320398-1,AFILIADO,MARIO ADOLFO,ARIZTIA,MONTANE,ARIZTIA MONTANE,COTIZANTE,M,1945-02-20,2021-10-29,2021-12-01,0.1624,53203981


In [76]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21772\1486029825.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_21772\1486029825.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [77]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(709, 27)

In [78]:
Pacientes_con_id.head()

,Rut Titular,DV Titular,Rut B.,DV B.,rut1,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,16644184-6,6,16644184-6,6,16644184-6,AFILIADO,Teresa De,LAGOS,SALDIVIA,Lagos Saldivia,...,True,0018c00002eimgCAAQ,166441846,TERESA DE LAGOS SALDIVIA,RUT,None,None,None,Mujer,None
1,2589337-9,9,2589337-9,9,2589337-9,AFILIADO,Gregoria,VELASCO,MINGUEZ,Velasco Minguez,...,True,0018c00002amSgPAAU,25893379,Gregoria Velasco Minguez,RUT,None,None,None,Mujer,995468783
2,4182163-9,9,4182163-9,9,4182163-9,AFILIADO,Luz Maria,SANHUEZA,GONZALEZ,Sanhueza Gonzalez,...,True,0018c00002ehrRhAAI,41821639,Luz Maria Sanhueza Gonzalez,RUT,None,None,None,Mujer,None
3,5289238-4,4,5289238-4,4,5289238-4,AFILIADO,Domingo,TORRES,TORREALBA,Torres Torrealba,...,True,0018c00002eVdXjAAK,52892384,Domingo Rodolfo Torres Torrealba,RUT,None,None,None,Hombre,None
4,5320398-1,1,5320398-1,1,5320398-1,AFILIADO,Mario Adolfo,ARIZTIA,MONTANE,Ariztia Montane,...,True,0018c00002amVkDAAU,53203981,Mario Adolfo Ariztia Montané,RUT,None,None,None,Hombre,None


In [79]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("Nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
56,False,Otilia Edit,Lucero Contreras,47569532,1942-11-24,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
57,False,Jessica,Sandoval Lucero,104336361,1965-02-17,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
58,False,Sebastian,Salas Sandoval,202955959,1999-07-12,Hombre,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
59,False,Daniela,Salas Sandoval,209518996,2002-01-07,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
61,False,Javier,Leal Testa,280633364,2023-02-07,Hombre,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [80]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [81]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
157180,a1Y8c00000C6UcaEAF,0018c00002amOpnAAE,a1W8c0000083j8rEAA,Colmena,0018c00002cXclQAAS,Colmena S.A.,Sin Holding Asociado,Colmena,Colmena S.A.,Sin Póliza Asociada,None,EMP6471,A-2317
157181,a1Y8c00000C6UcbEAF,0018c00002amOpsAAE,a1W8c0000083j8rEAA,Colmena,0018c00002cXclQAAS,Colmena S.A.,Sin Holding Asociado,Colmena,Colmena S.A.,Sin Póliza Asociada,None,EMP6471,A-2317
157182,a1Y8c00000C6UccEAF,0018c00002amOqCAAU,a1W8c0000083j8rEAA,Colmena,0018c00002cXclQAAS,Colmena S.A.,Sin Holding Asociado,Colmena,Colmena S.A.,Sin Póliza Asociada,None,EMP6471,A-2317
157183,a1Y8c00000C6UcdEAF,0018c00002amOqSAAU,a1W8c0000083j8rEAA,Colmena,0018c00002cXclQAAS,Colmena S.A.,Sin Holding Asociado,Colmena,Colmena S.A.,Sin Póliza Asociada,None,EMP6471,A-2317
157184,a1Y8c00000C6UceEAF,0018c00002amOqTAAU,a1W8c0000083j8rEAA,Colmena,0018c00002cXclQAAS,Colmena S.A.,Sin Holding Asociado,Colmena,Colmena S.A.,Sin Póliza Asociada,None,EMP6471,A-2317


In [82]:
campaña_seleccionada.shape

(1157, 13)

In [83]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [84]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [85]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
